# Identifikace systému, odhad parametrů modelu

Tento notebook je interaktivní, každou buňku můžete vykonat stisknutím `Shift-Enter`, můžete změnit její obsah a opětovně ji vykonat.

V kódu jsou tzv. elipsy, buď jako tři tečky `...` nebo podtržítko `_` v místě, kdě je potřeba doplnit nebo naprogramovat část dle zadání úkolu.

## Příprava v Modelice

- na fyzické mašině, 
- update `Bodylight-notebooks`, pomocí 
```
cd Bodylight-notebooks
git pull
```

- pokud nemáte spuštěnou virtuální mašinu, spusťte ji
```
cd Bodylight-VirtualMachine
(git pull)
(cd bodylightvm.20.11) # pokud máte nainstalovanou variantu z binárního celého obrazu
vagrant up
```

- na fyzické mašině otevřete ve svém prohlížeči (Firefox, Chrome, MS Edge, ...) adresu http://localhost:8080/jupyter
- otevřete JUPYTER notebook `Seminar8FmiIdentification/ModelicaIdentification2HW.ipynb` a splňte úkoly v sekci 5. Můžete odevzdat jako ipynb soubor `File->Download as->Notebook (ipynb)`.


## 1 Problém

Budeme zpracovávat data měření tzv. glukózového tolerančního testu. Máme informace, že pacient dostával intravenózně glukózu do krve rychlostí 28 mg/s. po dobu 12 minut. Pak byly v 12 minutových intervalech prováděny testy koncentrace glukózy a inzulínu hodnoty jsou v souborech `data/PatientX_y.csv` kde `X` je označení pacienta a `y` je buď `g`lukóza nebo `i`nzulín.

Úkol vyhodnoťte pacienty D,E,F, odhadněte parametry beta a nu, které nejlépe odpovídají datům.
Použijte algoritmy "differential_evolution" a "monte_carlo".


## 2 Načtení a vizuální kontrola dat

Spusťte buňky pomocí `Shift+Enter`. Všimněte si, že každý pacient má jiné hodnoty glukózy a inzulínu během glukózového tolerančního testu.

In [ ]:
# definice funkce pro vykresleni 2 grafu vedle sebe v jednom x,y,z v druhem x,y2,z2, z a z2 nemusi byt definovane
def plot2(x,y,z=None,y2=None,z2=None, labely=None,labelz=None,title1='glucose concentration',title2='insulin concentration'):
    %matplotlib inline
    import matplotlib.pyplot as plt   
    plt.rcParams["figure.figsize"] = (12, 6)
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(x,y,label=labely,color='red')
    if z is not None:
        ax1.plot(x,z, label=labelz,color='blue')
    ax1.set(title=title1)
    ax1.legend()
    ax1.grid(True)
    ax2.plot(x,y2,label=labely,color='red')
    if z2 is not None:
        ax2.plot(x,z2, label=labelz,color='blue')
    ax2.set(title=title2)
    ax2.legend()
    ax2.grid(True)
    plt.show()

In [ ]:
# data pacienta D
# ukol 1.1 doplnte cestu k souborum pro glukozu 'data/PatientD_g.csv' a pro inzulin 'data/PatientD_i.csv'
import numpy
dataDg = numpy.genfromtxt('data/PatientD_g.csv',delimiter=',')
dataDi = numpy.genfromtxt('data/PatientD_i.csv',delimiter=',')
time = numpy.arange(0,720*dataDg.size,720)
plot2(time,dataDg,y2=dataDi,labely='data pacienta D')

In [ ]:
# data pacienta E

dataEg = numpy.genfromtxt('data/PatientE_g.csv',delimiter=',')
dataEi = numpy.genfromtxt('data/PatientE_i.csv',delimiter=',')
#time = numpy.arange(0,720*dataDg.size,720)
plot2(time,dataEg,y2=dataEi,labely='data pacienta E')

In [ ]:
# data pacienta F
dataFg = numpy.genfromtxt('data/PatientF_g.csv',delimiter=',')
dataFi = numpy.genfromtxt('data/PatientF_i.csv',delimiter=',')
plot2(time,dataFg,y2=dataFi,labely='data pacienta F')

## 3 Definice pomocných rutin a funkcí

Budeme používat knihovnu pyfmi. Pokud následující buňka proběhne bez chyb, pokračujte dál. Pokud se vyskytne chyba, pak musíte nainstalovat pyfmi: ve virtuální mašině nastartujte terminál a zadejte `conda install -c conda-forge pyfmi`. p úspěšné instalaci zkuste spustit opět tuto buňku.

In [ ]:
from pyfmi import load_fmu

Definice funkcí pro simulaci s parametry

In [ ]:
# simulace s nastavenim 2 parametru a nastaveni pyfmi
model1 = 'fmus/Metabolism.GlucoseInsulin.Test.GlucoseToleranceTest.fmu'
modelp = load_fmu(model1)
opts = modelp.simulate_options()
opts["ncp"] = 360
opts["CVode_options"]["store_event_points"] = False # by default it is true
def simulatemodelpyfmi(relbeta=1,
                       relnu=1,
                       betaname='glucoseInsulinRegulation.insulinProduction.beta_mu',
                       nuname='glucoseInsulinRegulation.tissueUtilizationInsulinDependent.Nu_permu',
                       sopts=opts):        
    modelp.reset()
    modelp.instantiate()
    modelp.set(nuname,139000*relnu)
    modelp.set(betaname, 1430*relbeta)
    modelp.set("glucoseInfusion.width",20)
    res = modelp.simulate(final_time=3*24*60*60,options = sopts) 
    return res

In [ ]:
# funkce pro vizuální provnání dat s modelem
cindex=249 # index kalibrace, viz seminář
def compare2pyfmimodeldata(relativebeta,relativenu,var1,var2,param1,param2,realdata=dataDg,realdata2=dataDg,realtime=time,modelfile=model1):
    modelresult = simulatemodelpyfmi(relativebeta,relativenu,param1,param2)
    # vykreslit do grafu
    plot2(realtime,
         realdata,modelresult[var1][cindex:cindex+10],
         realdata2,modelresult[var2][cindex:cindex+10],
         labely='real data',labelz='model')

In [ ]:
# definice míry vzdálenosti dat od simulace modelu pro 2 datove sady s1,s2 simulace, d1,d2 data
def mydiff2(s1,d1,s2,d2):
    sum = 0
    for i in range(0,s1.size):
        sum+=((s1[i]-d1[i])*10)**2+((s2[i]-d2[i])*100)**2
    return sum

In [ ]:
# metoda monte-carlo pro parametry
def montecarlo(datag,datai,varname1,varname2,param1,param2,cindex=249,iterations=50):
    import random
    #datacg = numpy.genfromtxt("data/PatientB_g.csv")
    #dataci = numpy.genfromtxt("data/PatientB_i.csv")
    #cindex=249
    diffs = []
    for i in range(1,iterations):
        b=random.random(); # generuje nahodne cislo mezi 0 a 1
        n=random.random(); # generuje nahodne cislo mezi 0 a 1        
        modelresult = simulatemodelpyfmi(b,n,param1,param2)
        diff= mydiff2(modelresult[varname1][cindex:cindex+10],datag,
                     modelresult[varname2][cindex:cindex+10],datai)
        diffs.append([b,n,diff])
    return diffs

In [ ]:
# definice funkce tak jak ji ocekava algoritmus diferential evolution 
# obdrzi pole hodnot parametru a musi vrati miru rozdilu mezi simulovanymi a realnymi daty
# jmena sledovanych promennych a data jsou v globalnich promennych param1,param2 a var1,var2 a data1,data2
def objectiveFunctionForGIR(p):
    r = simulatemodelpyfmi(p[0],p[1],param1,param2)
    diff= mydiff2(r[var1][cindex:cindex+10],data1,
                 r[var2][cindex:cindex+10],data2)
    return diff

In [ ]:
# nastaveni globalnich promennych 
var1="glucoseInsulinRegulation.glucoseConc"
var2="glucoseInsulinRegulation.insulinConc"
param1="glucoseInsulinRegulation.insulinProduction.beta_mu"
param2="glucoseInsulinRegulation.tissueUtilizationInsulinDependent.Nu_permu"

## 4 odhad parametrů

Data máme načtená v polích
```
# pro pacienta D: dataDg dataDi
# pro pacienta E: dataEg dataEi
# pro pacienta F: dataFg dataFi
```

### 4.1 Odhadneme parametry metodou Monte-Carlo

s 1000 iteracemi (1000x se zavolá simulace pro rozdílné parametry) pro pacienta D.

In [ ]:
# odhad parametru metodou monte carlo - dosadim data pacienta D - dataDg a dataDi
diffsD = montecarlo(dataDg,dataDi,var1,var2,param1,param2,iterations=1000)

In [ ]:
diffsD.sort(key=lambda x:x[2]) 
compare2pyfmimodeldata(diffsD[0][0],diffsD[0][1],var1,var2,param1,param2,realdata=dataDg,realdata2=dataDi)
print('nejlepsi hodnoty parametru beta:',diffsD[0][0],'nu:',diffsD[0][1],' diff:',diffsD[0][2])

### 4.2 Odhadneme parametry metodou Differential-Evolution

pro pacienta D. Dokumentace viz https://docs.scipy.org/doc/scipy/reference/optimize.html

In [ ]:
# importujeme knihovnu scipy
import scipy.optimize

In [ ]:
# nastavíme data1 a data2 pro objectiveFunctionForGIR na data našeho pacienta
data1=dataDg
data2=dataDi
resD = scipy.optimize.differential_evolution(objectiveFunctionForGIR, [[0,1],[0,1]])

In [ ]:
# výpis výsledku
resD

In [ ]:
compare2pyfmimodeldata(resD.x[0],resD.x[1],var1,var2,param1,param2,realdata=data1,realdata2=data2)
print('nejlepsi hodnoty parametru beta:',resD.x[0],'nu:',resD.x[1],' diff:',resD.fun)

## 5 Odhad parametrů a srovnání výsledků
  * 5.1 Proveďte odhad parametrů pro pacienta E a F oběma metodami (Monte-Carlo a Differential-Evolution)
  * 5.2 Určete počet iterací které provedl algoritmus differential_evolution (struktura resD), dokumentace viz https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution)
  * 5.3 Srovnejte výsledky odhadů (beta,nu), míry rozdílu od reálných dat (diff) a počet iterací (simulací, které metoda provedla) obou metod pro všechny pacienty D,E,F (např. tabulkou)
